# Needleman–Wunsch

In [1]:
import numpy as np

def match_or_mismatch(x, mu, char_a, char_b):
    return x + (char_a == char_b) - mu * (char_a != char_b)

def gap(x, delta):
    return x - delta

def simple_alignments(s, t, delta, mu):
    d = np.zeros((len(s) + 1, len(t) + 1))
    d[0] = np.arange(len(t) + 1) * (-delta)
    
    for i in range(1, len(s) + 1):
        d[i][0] = - delta * i
        for j in range(1, len(t) + 1):
            d[i][j] = max(
                gap(d[i-1][j], delta),
                gap(d[i][j-1], delta),
                match_or_mismatch(d[i-1][j-1], mu, s[i-1], t[j-1])
            )

    s1 = ''
    t1 = ''
    i = len(s)
    j = len(t)
    while (i > 0)and(j > 0):
        if abs(match_or_mismatch(d[i-1][j-1], mu, s[i-1], t[j-1]) 
               - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += t[j-1]
            i -= 1
            j -= 1
        elif abs(gap(d[i][j-1], delta) 
                 - d[i][j]) < 1e-10:
            s1 += '-'
            t1 += t[j-1]
            j -= 1
        elif abs(gap(d[i-1][j], delta) 
                 - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += '-'
            i -= 1

    while (i > 0):
        s1 += s[i-1]
        t1 += '-'
        i -= 1
    while (j > 0):
        s1 += '-'
        t1 += t[j-1]
        j -= 1
        
    return s1[::-1], t1[::-1]

In [2]:
# TEST1

mu = 1
delta = 1

s = 'AGTGGCATTACATGCGCTA'
t = 'CGTACTCAGCATCTGGCAC'
    
print('{}\n{}'.format(*simple_alignments(s, t, delta, mu)))

AGTGGCATTA-CATGC--GCTA-
CGT-AC-TCAGCAT-CTGGC-AC


In [3]:
# TEST2

mu = 1
delta = .499
    
print('{}\n{}'.format(*simple_alignments(s, t, delta, mu)))

A-GTGGCAT-T-A-CATGC--GCTA-
-CGT---A-CTCAGCAT-CTGGC-AC


# With matrix

In [4]:
def calculate_weight(x, y, weight_matrix):
    letter_to_num = {'A': 0, 'C': 1, 'G': 2, 'T': 3, '-': 4}
    row = letter_to_num[x]
    column = letter_to_num[y]
    if row > column:
        row, column = column, row
    return weight_matrix[row][column]

def alignment(s, t, weight_matrix):
    d = np.zeros((len(s) + 1, len(t) + 1))
    d[0] = np.arange(len(t) + 1) * (-delta)
    
    for i in range(1, len(s) + 1):
        d[i][0] = - delta * i
        for j in range(1, len(t) + 1):
            d[i][j] = max(
                gap(d[i-1][j], calculate_weight(s[i-1], '-', weight_matrix)),
                gap(d[i][j-1], calculate_weight(t[j-1], '-', weight_matrix)),
                match_or_mismatch(d[i-1][j-1], 
                                  calculate_weight(s[i-1], t[j-1], weight_matrix),
                                  s[i-1], t[j-1])
            )

    s1 = ''
    t1 = ''
    i = len(s)
    j = len(t)
    while (i > 0)and(j > 0):
        if abs(match_or_mismatch(d[i-1][j-1], 
                                 calculate_weight(s[i-1], t[j-1], weight_matrix), 
                                 s[i-1], t[j-1]) 
               - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += t[j-1]
            i -= 1
            j -= 1
        elif abs(gap(d[i][j-1], calculate_weight(t[j-1], '-', weight_matrix)) 
                 - d[i][j]) < 1e-10:
            s1 += '-'
            t1 += t[j-1]
            j -= 1
        elif abs(gap(d[i-1][j], calculate_weight(s[i-1], '-', weight_matrix)) 
                 - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += '-'
            i -= 1

    while (i > 0):
        s1 += s[i-1]
        t1 += '-'
        i -= 1
    while (j > 0):
        s1 += '-'
        t1 += t[j-1]
        j -= 1
        
    return s1[::-1], t1[::-1]

In [5]:
np.random.seed(42)
weight_matrix = np.random.randint(0, 5, 25).reshape((5, 5))

s = 'ACACGCTTCGAAGTC'
t = 'CCGCTCATGCACGTA'

print('1: {}\n   {}'.format(*alignment(s, t, weight_matrix)))

weight_matrix[1][4] -= 2

print('2: {}\n   {}'.format(*alignment(s, t, weight_matrix)))

1: ACACGC-TTCGAAGTC
   CCGCTCATGC-ACGTA
2: ACA-CGCT--TCG-AAGTC
   ---CCGCTCAT-GCACGTA


# Smith–Waterman

In [6]:
def local_alignments(s, t, delta, mu):
    d = np.zeros((len(s) + 1, len(t) + 1))
    d[0] = np.arange(len(t) + 1) * (-delta)
    
    max_element = -1
    max_index = (-1, -1)
    for i in range(1, len(s) + 1):
        d[i][0] = - delta * i
        for j in range(1, len(t) + 1):
            d[i][j] = max(
                0,
                gap(d[i-1][j], delta),
                gap(d[i][j-1], delta),
                match_or_mismatch(d[i-1][j-1], mu, s[i-1], t[j-1])
            )
            if d[i][j] > max_element:
                max_element = d[i][j]
                max_index = (i, j)


    i = max_index[0]
    j = max_index[1]
    s1 = '-' * max(len(t) - j - len(s) + i, 0) + s[:i-1:-1].lower()
    t1 = '-' * max(len(s) - i - len(t) + j, 0) +t[:j-1:-1].lower()
    while (i > 0)and(j > 0):
        if abs(d[i][j]) < 1e-10:
            break
        elif abs(match_or_mismatch(d[i-1][j-1], mu, s[i-1], t[j-1]) 
                 - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += t[j-1]
            i -= 1
            j -= 1
        elif abs(gap(d[i][j-1], delta) 
                 - d[i][j]) < 1e-10:
            s1 += '-'
            t1 += t[j-1]
            j -= 1
        elif abs(gap(d[i-1][j], delta) 
                 - d[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += '-'
            i -= 1

    while (i > 0)and(d[i][j] > 0):
        s1 += s[i-1]
        t1 += '-'
        i -= 1
    while (j > 0)and(d[i][j] > 0):
        s1 += '-'
        t1 += t[j-1]
        j -= 1
    s1 += s[i-1::-1].lower() + '-' * max(j - i, 0)
    t1 += t[j-1::-1].lower() + '-' * max(i - j, 0)
        
    return s1[::-1], t1[::-1]

In [7]:
# TEST

mu = 1
delta = 1

s = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
t = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

print('Global:\n{}\n{}'.format(*simple_alignments(s, t, delta, mu)))
print('\nLocal:\n{}\n{}'.format(*local_alignments(s, t, delta, mu)))

Global:
---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC

Local:
------------------tccCAGTTATGTCAGgggacacgagcatgcagagac
aattgccgccgtcgttttcagCAGTTATGTCAGatc------------------


# Global:
---T-CC-<font color="red">CAGT--TATGTCAG</font>GGGACACGAGCATG-CAGAGAC <br>
AATTGCCGCCGTCGT-TTTCA---G-<font color="red">CA-G-TTATGTCAG</font>A-TC
## Local:
-------------------------------TCC<font color="red">CAGTTATGTCAG</font>GGGACACGAGCATGCAGAGAC<br>
AATTGCCGCCGTCGTTTTCAG<font color="red">CAGTTATGTCAG</font>ATC

# Affine transformation

In [8]:
def affine_alignments(s, t, delta, p, sigma):
    lower_level = -np.ones((len(s) + 1, len(t) + 1)) * p
    main_level = -np.ones((len(s) + 1, len(t) + 1)) * p
    upper_level = -np.ones((len(s) + 1, len(t) + 1)) * p
    
    for level in [lower_level, main_level, upper_level]:
        level[0] -= np.arange(len(t) + 1) * sigma
        level[:, 0] -= np.arange(len(t) + 1) * sigma
        level[0][0] = 0
    
    for i in range(1, len(s) + 1):
        for j in range(1, len(t) + 1):
            lower_level[i][j] = max(gap(main_level[i-1][j], p+sigma), 
                                    gap(lower_level[i-1][j], sigma))
            
            upper_level[i][j] = max(gap(main_level[i][j-1], p+sigma), 
                                    gap(upper_level[i][j-1], sigma))
            
            main_level[i][j] = max(match_or_mismatch(main_level[i-1][j-1], delta, s[i-1], t[j-1]), 
                                    upper_level[i][j], lower_level[i][j])

    s1 = ''
    t1 = ''
    i = len(s)
    j = len(t)
    while (i > 0)and(j > 0):
        if abs(match_or_mismatch(main_level[i-1][j-1], delta, s[i-1], t[j-1]) 
               - main_level[i][j]) < 1e-10:
            s1 += s[i-1]
            t1 += t[j-1]
            i -= 1
            j -= 1
        elif (abs(gap(upper_level[i][j-1], sigma) - main_level[i][j]) < 1e-10) \
        or(abs(gap(main_level[i][j-1], p+sigma) - main_level[i][j]) < 1e-10):
            s1 += '-'
            t1 += t[j-1]
            j -= 1
        elif (abs(gap(lower_level[i-1][j], sigma) - main_level[i][j]) < 1e-10) \
        or(abs(gap(main_level[i-1][j], p+sigma) - main_level[i][j]) < 1e-10):
            s1 += s[i-1]
            t1 += '-'
            i -= 1

    while (i > 0):
        s1 += s[i-1]
        t1 += '-'
        i -= 1
    while (j > 0):
        s1 += '-'
        t1 += t[j-1]
        j -= 1
        
    return s1[::-1], t1[::-1]

In [14]:
# TEST

s = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
t = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

delta, p, sigma = 1, 0, 1 #  Needleman–Wunsch
print('1: {}\n   {}'.format(*affine_alignments(s, t, delta, p, sigma)))

delta, p, sigma = 1, 100, .01 #  No gaps
print('2: {}\n   {}'.format(*affine_alignments(s, t, delta, p, sigma)))

delta, p, sigma = 1, -0.5, .3
print('3: {}\n   {}'.format(*affine_alignments(s, t, delta, p, sigma)))

0: Needleman–Wunsch
   ---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
   AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC

1: ---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
   AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC
2: TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
   AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC
3: --T--CC-CA-GT--TATG-TCAGGGGACACGAGC--ATG-CAGAGA-C
   AATTGCCGC-CGTCGT-T-TTCA---G---C-AG-TTATGTC--AGATC
